In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

from config import DATA_FOLDER

BASE_PATH = Path("..")
data_path = BASE_PATH.joinpath(DATA_FOLDER)
tdd_path = data_path / "MVTecAD" / "wood"

### Unsupervised Train

In [2]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if img is not None:
            images.append(img)
    return np.asarray(images)

# read training images
x_train = load_images_from_folder(tdd_path / "train" / "good")
y_train = np.zeros(x_train.shape[0])

In [3]:
x_train.shape, y_train.shape

((247, 1024, 1024, 3), (247,))

In [4]:
np.savez(tdd_path / 'train', x=x_train, y=y_train)

### Supervised Test

In [5]:
x_test = []
y_test = []

for root, dirs, files in os.walk(tdd_path / "test"):
    for file in files:
        img = cv2.imread(str(Path(root) / Path(file)))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x_test.append(img)
        y_test.append(Path(root).name)

x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

In [6]:
x_test.shape, y_test.shape

((79, 1024, 1024, 3), (79,))

In [7]:
classes = ['good','color','hole','liquid','scratch','combined']

def convert_labels(labels, classes):
    return [classes.index(label) for label in labels]

In [8]:
y_test = convert_labels(y_test, classes)

In [9]:
np.savez(tdd_path / 'test', x=x_test, y=y_test)